In [3]:

import logging
from dotenv import load_dotenv

from typing import Annotated
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
load_dotenv()

from metagame import TradingClient
from metagame.websocket_api import Side
from typing import Literal
from metagame.websocket_api import CancelOrder, ClientMessage, CreateOrder, Side

from os import getenv
import time

JWT = getenv("JWT")
API_URL = getenv("API_URL") 
# ACT_AS = int(getenv("ACT_AS", None))
ACT_AS=106 # arrbot



ETFS = {
    'ghi_tw': {
        'golf_tw': 3,
        'hotel_tw': 2,
        'india_tw': 1
    },
    'def_tw': {
        'delta_tw': 1,
        'echo_tw': 1,
        'foxtrot_tw': 4
    },
    'abc_tw': {
        'alpha_tw': 2,
        'bravo_tw': 2,
        'charlie_tw': 2
    }
}

ALL_MARKETS = list(set(ETFS.keys()).union(*[set(etf_constituents.keys()) for etf_constituents in ETFS.values()]))


def get_price_size(market, market_type: Literal['etf', 'stock'], considering_etf_bid: bool):
    '''returns best price, size. Returns None, 0 if no contract available'''
    assert market_type in ['etf', 'stock']
    considering_bid = considering_etf_bid if market_type == 'etf' else not considering_etf_bid
    
    if considering_bid:
        if len(market.offers) == 0:
            return None, 0
        return market.offers[0].price, market.offers[0].size
    else:
        if len(market.bids) == 0:
            return None, 0
        return market.bids[0].price, market.bids[0].size
    

In [ ]:
while True:
    time.sleep(0.1)
    with TradingClient(API_URL, JWT, ACT_AS) as client:
        state = client.state()
        market_name_to_id = state.market_name_to_id

        name_to_market = {}
        for market_name in ALL_MARKETS:
            market_id = state.market_name_to_id[market_name]
            name_to_market[market_name] = state.markets[market_id]

        for etf_name in ETFS:
        # etf_name = 'abc_tw'
            etf_constituents = ETFS[etf_name]
            etf_stock_names = list(etf_constituents.keys())
            etf_market = name_to_market[etf_name]

            for considering_etf_bid in [True, False]:
                state = client.state()
                # get etf contracts (bid by default)
                etf_price, etf_size = get_price_size(market=etf_market, market_type='etf', considering_etf_bid=considering_etf_bid)

                unbundled_price = 0
                etf_deal_size = etf_size # the bottleneck of the deal

                stock_prices = {}
                for stock_name in etf_stock_names:
                    stock = name_to_market[stock_name]
                    price, size = get_price_size(market=stock, market_type='stock', considering_etf_bid=considering_etf_bid)

                    stock_weight = etf_constituents[stock_name]

                    # etf_weight_deal_size is the size of the best bid/offer, weighted by the stock weight
                    etf_weight_deal_size = size/stock_weight
                    etf_deal_size = min(etf_deal_size, etf_weight_deal_size)
                    
                    unbundled_price += stock_weight*price

                    stock_prices[stock_name] = price


                if (considering_etf_bid and etf_price < unbundled_price) or (not considering_etf_bid and etf_price > unbundled_price):
                    if considering_etf_bid:
                        assert etf_price < unbundled_price
                    else:
                        assert etf_price > unbundled_price
                    
                    if (etf_price - unbundled_price).abs() < 2.0:
                        continue

                    # # do the deal
                    # orders = []
                    # for stock_name in etf_stock_names:
                    #     orders.append(
                    #         ClientMessage(
                    #             create_order=CreateOrder(
                    #             market_id=market_name_to_id[stock_name],
                    #             price=stock_prices[stock_name],
                    #             size=etf_deal_size*etf_constituents[stock_name],
                    #             side=Side.OFFER if considering_etf_bid else Side.BID,
                    #             )
                    #     ))
                    # orders.append(
                    #     ClientMessage(
                    #         create_order=CreateOrder(
                    #             market_id=market_name_to_id[etf_name],
                    #             price=(bundled_price + unbundled_price)/2,
                    #             size=etf_deal_size,
                    #             side=Side.BID if considering_etf_bid else Side.OFFER,
                    #         )
                    #     )
                    # )
#                     s = f'''Considering doing this deal:
# {"ETF BID" if considering_etf_bid else "ETF OFFER"}

# ETF
# {etf_name}: {etf_price}

# STOCKS
# '''
#                     for stock_name in etf_stock_names:
#                         s += f"{stock_name}: {stock_prices[stock_name]}\n"
                    
#                     s += f'''\nBUNDLED PRICE: {etf_price}
# UNBUNDLED PRICE: {unbundled_price}

# DEAL SIZE: {etf_deal_size}
                    
# '''
#                     time.sleep(1)


                    

                    # logger.info(f"Placing {len(orders)} orders")

                    # try:
                    #     client.request_many(orders)
                    #     time.sleep(0.5)

                    # except Exception as e:
                    #     logger.error(f"Error placing orders: {e}")
                    #     continue
                else:
                    # logger.info(f"No deal to do in {etf_name}")
                    continue




Considering doing this deal:
ETF BID

ETF
def_tw: 703.5

STOCKS
delta_tw: 112.0
echo_tw: 139.5
foxtrot_tw: 113.5

BUNDLED PRICE: 703.5
UNBUNDLED PRICE: 705.5

DEAL SIZE: 0.2
                    

Considering doing this deal:
ETF BID

ETF
def_tw: 705.0

STOCKS
delta_tw: 112.0
echo_tw: 139.5
foxtrot_tw: 113.5

BUNDLED PRICE: 705.0
UNBUNDLED PRICE: 705.5

DEAL SIZE: 0.6
                    

Considering doing this deal:
ETF BID

ETF
def_tw: 705.0

STOCKS
delta_tw: 112.0
echo_tw: 139.5
foxtrot_tw: 113.5

BUNDLED PRICE: 705.0
UNBUNDLED PRICE: 705.5

DEAL SIZE: 0.4
                    

Considering doing this deal:
ETF BID

ETF
def_tw: 705.0

STOCKS
delta_tw: 112.0
echo_tw: 139.5
foxtrot_tw: 113.5

BUNDLED PRICE: 705.0
UNBUNDLED PRICE: 705.5

DEAL SIZE: 0.1
                    

Considering doing this deal:
ETF BID

ETF
abc_tw: 720.0

STOCKS
alpha_tw: 108.0
bravo_tw: 151.0
charlie_tw: 117.0

BUNDLED PRICE: 720.0
UNBUNDLED PRICE: 752.0

DEAL SIZE: 0.9
                    

Considering doing t

KeyboardInterrupt: 

NameError: name 'market_exposures' is not defined

In [21]:
with TradingClient(API_URL, JWT, ACT_AS) as client:
    state = client.state()
    # print(state.market_exposures)
    # print(state.market_exposures)
    print(dir(state))
    print(state.portfolio.market_exposures[0])

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_initializing', '_update', 'accounts', 'acting_as', 'market_name_to_id', 'markets', 'portfolio', 'portfolios', 'transfers', 'user_id']
PortfolioMarketExposure(market_id=88, position=0.0, total_bid_size=0.0, total_offer_size=0.0, total_bid_value=0.0, total_offer_value=0.0)


In [ ]:
# Considering doing this deal:
# ETF BID

# ETF
# def_tw: 703.5

# STOCKS
# delta_tw: 112.0
# echo_tw: 139.5
# foxtrot_tw: 113.5

# BUNDLED PRICE: 703.5
# UNBUNDLED PRICE: 705.5

# DEAL SIZE: 0.2

In [ ]:
!pip3 install dotenv